In [337]:
import numpy as np
from scipy.stats import uniform, norm, f, t
import statistics
from scipy import linalg
from sklearn.linear_model import LinearRegression
import random
from sklearn.model_selection import cross_validate

In [338]:
N = 50
x = np.zeros((6,50))
a = np.zeros(50)
for i in range(5):
    x[i,:] = uniform.rvs(loc=-1, scale=2, size = 50)
for j in range(50):
    a[j] = 2 + 3*x[0,j] - 2*x[1,j] + x[2,j] + x[3,j] - x[4,j]
    x[5,j] = norm.rvs(a[j], 1.5)

## a)

In [339]:
x_ksi = x[:5,:]
R_sq = np.zeros(5)
for i in range(5):
    y = x_ksi[i, :]
    TSS = np.sum(np.square(y - np.mean(y)))
    
    psi = np.ones((50,5))
    psi[:,1:] = np.delete(x_ksi,i,0).T
    F_inv = np.linalg.inv(psi.T@psi)
    beta = F_inv@psi.T@y
    RSS = (y - psi@beta).T @ (y - psi@beta)
    
    R_sq[i] = 1 - RSS/TSS
R_sq

array([0.05547866, 0.06955145, 0.01900459, 0.01461188, 0.09376939])

=> ξ не являются мультиколлинеарными

## b)

In [340]:
# "вручную" (β[0] - у нас последний коэффициент)
target_eta = np.array([x[5,0], x[5,12], x[5,19], x[5,25], x[5,38], x[5,49]])
target_ksi = np.ones((6,6))
for i in range(5):
    target_ksi[i, :] = np.array([x[i,0], x[i,12], x[5,19], x[i,25], x[i,38], x[i,49]])
betas_hand = linalg.solve(target_ksi, target_eta)
betas_hand

array([ 2.33656904, -2.65579625,  0.31028481, -1.05338692,  2.74274886,
        2.12574325])

In [341]:
# с помощью встроенной библиотеки для линейной регресси, без нулевого веса
x_reg = x[:5,:].T
y_reg = x[5,:]
reg = LinearRegression().fit(x_reg, y_reg)
betas = reg.coef_
betas

array([ 2.89952826, -1.82314712,  1.04053171,  0.92765595, -1.1462501 ])

In [342]:
# проверяем значимость бутстрапом (можно было не бутстрапом, а через t(N-6))
delta_boot = np.zeros((1000,5))
y = np.zeros((6,50))
for i in range(1000):
    y[0, :] = np.array(random.sample((x[0,:]).tolist(), N, counts = [100 for i in range(N)])) #хотим двумерную подвыборку или два цикла
    for m in range(50):
        for k in range(50):
            if x[0, k] == y[0, m]:
                y[:, m] = x[:, k]
                break
        
    x_boot = y[:5,:].T
    y_boot = y[5,:]
    reg_boot = LinearRegression().fit(x_boot, y_boot)
    betas_boot = reg_boot.coef_
    delta_boot[i, :] = np.abs(betas_boot - betas)

In [343]:
for i in range(5):
    m = sum(np.sort(delta_boot[:, i])>=abs(betas[i]))
    p_value = m/1000
    print(p_value)
    if p_value > 0.05:
        print('нет оснований отвергнуть гипотезу о незначимости коэффициента beta', i+1, sep='')
    else:
        print('отвергаем гипотезу о незначимости коэффициента beta', i+1, sep='')

0.0
отвергаем гипотезу о незначимости коэффициента beta1
0.0
отвергаем гипотезу о незначимости коэффициента beta2
0.002
отвергаем гипотезу о незначимости коэффициента beta3
0.001
отвергаем гипотезу о незначимости коэффициента beta4
0.002
отвергаем гипотезу о незначимости коэффициента beta5


In [344]:
# сделаем не через бутстрап
for i in range(5):
    deltas = betas[i]/np.sqrt(RSS*F_inv[i,i])*(N-6)**0.5
    p_values = 2*(1 - t.cdf(abs(deltas), N-6))
    print(p_values)
    if p_values > 0.05:
        print('нет оснований отвергнуть гипотезу о незначимости коэффициента beta', i+1, sep='')
    else:
        print('отвергаем гипотезу о незначимости коэффициента beta', i+1, sep='')

0.0
отвергаем гипотезу о незначимости коэффициента beta1
6.17284001691587e-14
отвергаем гипотезу о незначимости коэффициента beta2
2.7280707470822563e-08
отвергаем гипотезу о незначимости коэффициента beta3
2.9759530706385817e-07
отвергаем гипотезу о незначимости коэффициента beta4
5.1179480653473775e-09
отвергаем гипотезу о незначимости коэффициента beta5


## c)

In [345]:
y = x[5, :]
TSS = np.sum(np.square(y - np.mean(y)))
    
psi = np.ones((50,6))
psi[:,1:] = np.delete(x,5,0).T
F_inv = np.linalg.inv(psi.T@psi)
beta = F_inv@psi.T@y
RSS = (y - psi@beta).T @ (y - psi@beta)
    
R_eta = 1 - RSS/TSS

print('коэффициент детерминации: ', R_eta, sep='')

коэффициент детерминации: 0.7646153986762193


In [346]:
delta_est = R_eta**2/(1-R_eta**2) * (N - 6)/(6-1)
delta_est

12.386272757412321

Δ ~ F(6-1,50-1) = F(5,49)

In [347]:
p_value = 1 - f.cdf(np.abs(delta_est), 6-1, N-1)
p_value

8.685118990658935e-08

=> отвергаем гипотезу о незначимости коэффициента детерминации

## d)

In [348]:
a_0 = 2
eta_0 = norm.rvs(a_0, 1.5)
print ('рандомное значение в нуле: ', eta_0, sep='')

рандомное значение в нуле: 1.0540166057619147


In [349]:
# построим численный доверительный интервал
delta_int = np.zeros(1000)
for i in range(1000):
    eta_boot = norm.rvs(a_0, 1.5)
    delta_int[i] = eta_boot - eta_0
print('доверительный интервал: (', np.sort(delta_int)[24], ', ', np.sort(delta_int)[974], ')', sep='')

доверительный интервал: (-1.9532612359347588, 3.8633486506199612)


## e)

In [350]:
y = x[5,:]
e = np.zeros(50)
psi = np.ones((50,6))
psi[:,1:] = np.delete(x,5,0).T
F_inv = np.linalg.inv(psi.T@psi)
beta = F_inv@psi.T@y
e = y - psi@beta
e

array([-0.24726988, -0.00890234, -0.61982416, -0.07147556,  0.43725623,
        0.38270074, -0.3592887 , -1.0187238 ,  1.83686518, -0.72188018,
        1.06837631,  1.07615762,  0.61361049, -0.5158088 , -0.3180563 ,
        1.81869033, -0.06289523,  1.34588503, -2.60249845,  0.81193918,
        0.00686419,  1.41271796, -0.60664883, -0.98532389,  0.18864711,
       -0.15424141,  1.14473752, -0.22017931,  3.46599444,  1.24786995,
        0.04072783, -0.45964323, -0.064818  , -1.34655458,  0.41399606,
        1.99932542, -2.3769679 ,  2.43521829, -0.72747095, -1.2086683 ,
       -1.02811503, -1.56477122, -2.97336894, -1.39377595,  0.15046273,
        0.76492712, -0.30241523,  0.39783542,  1.07396806, -2.17518704])

In [351]:
I=0
for i in range(N):
    for j in range (i+1, N):
        if(e[i] > e[j]):
            I += 1
I

674

Δ ~> N(0,1)

In [352]:
delta_est = (I - N*(N-1)/4)/(N**3/36)**0.5
p_value = 2*(1 - norm.cdf(abs(delta_est)))
p_value

0.29662901888552495

=> нет оснований отвергнуть гипотезу о независимости ошибок измерения

## g)

In [353]:
eps = np.zeros(50)
eps = x[5,:] - reg.predict(x_reg)

In [354]:
sigma = np.median(np.abs(eps))/0.675
sigma

1.105480053182846

In [373]:
quantity = 0
for i in range(50):
    if eps[i]<=-2*sigma or eps[i]>=2*sigma:
        quantity+=1
        print(i, ' является выбросом: eta = ', eps[i], sep='')
if quantity == 0:
    print('выбросов нет!')

18 является выбросом: eta = -2.6024984528218518
28 является выбросом: eta = 3.4659944402486875
36 является выбросом: eta = -2.3769679001347654
37 является выбросом: eta = 2.43521828533284
42 является выбросом: eta = -2.973368938862827


## f)

In [356]:
#Колмогоров

def F_est(x):
    F = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        F[i] = np.argsort(x)[i]/x.shape[0]
    return F

In [357]:
teta2_est = (np.sum(np.square(eps))/N)**0.5
print('оценка дисперсии: ', teta2_est, sep = '')

оценка дисперсии: 1.2694315525529103


In [358]:
delta_eps = N**0.5 * np.max(np.abs(F_est(eps) - norm.cdf(eps, 0, teta2_est)))
delta_eps

6.147791029959771

In [359]:
delta_boot = np.zeros(10000)
for i in range(10000):
    y = np.array(random.sample((eps).tolist(), N, counts = [100 for i in range(N)]))
    teta2_boot = (np.sum(np.square(y))/N)**0.5
    F = norm(0, teta2_boot)
    delta_boot[i] = N**0.5 * np.max(abs(F_est(eps) - F.cdf(eps)))
k = sum(delta_boot < delta_eps)+1
p_value = 1 - (k-1)/10000
p_value

0.5252

=> нет оснований отвергнуть гипотезу о нормальности распределения ошибок

## h)

In [360]:
cv = np.zeros(50)
for i in range(50):
    x_test = x[:5,i].reshape((5,1)).T
    y_test = x[5,i]
    x_train = np.delete(x[:5,:].T, i, 0)
    y_train = np.delete(x[5,:], i)
    reg_cv = LinearRegression().fit(x_train, y_train)
    cv[i] = (reg_cv.predict(x_test) - y_test)**2
cv

array([6.97986287e-02, 1.04462007e-04, 5.30353301e-01, 5.73792333e-03,
       2.22591887e-01, 2.19539753e-01, 1.61587795e-01, 1.43190490e+00,
       4.11431080e+00, 6.61199953e-01, 1.30927128e+00, 1.51148497e+00,
       5.30951624e-01, 3.25628289e-01, 1.27366085e-01, 4.00432770e+00,
       4.48905043e-03, 2.47615221e+00, 8.71858492e+00, 8.56464109e-01,
       6.90959878e-05, 2.82996394e+00, 4.92175404e-01, 1.10107773e+00,
       4.37807250e-02, 3.18576717e-02, 1.61527358e+00, 5.85999352e-02,
       1.56024703e+01, 2.12384481e+00, 2.17765425e-03, 2.73966781e-01,
       5.93438481e-03, 2.46231842e+00, 2.23601631e-01, 5.04018891e+00,
       8.21890074e+00, 6.68663289e+00, 6.00977300e-01, 1.99740939e+00,
       1.42629625e+00, 3.13151932e+00, 1.31682850e+01, 3.16158535e+00,
       2.76823216e-02, 7.32237655e-01, 1.25587284e-01, 1.85554474e-01,
       1.63045795e+00, 6.05723072e+00])

In [361]:
CVSS = np.sum(cv)
TSS = np.sum(np.square(x[5,:] - np.mean(x[5,:])))
R_cv = 1 - CVSS/TSS
R_cv

0.6893408782972705

## i)

In [362]:
# пусть все ξ = 0.5
RSS = (x[5,:] - psi@beta).T @ (x[5,:] - psi@beta)
b = 2 + 0.5*(3-2+1+1-1)
eta_ad = np.zeros(5)
for i in range(5):
    eta_ad[i] = norm.rvs(b, 1.5)
sigma_cap = 1/4*np.sum(np.square(eta_ad - np.mean(eta_ad)))
delta_est = RSS/((N-6)*sigma_cap)
delta_est

0.5192048451449349

In [363]:
p_value = 1 - f.cdf(np.abs(delta_est), N-6, 4)
p_value

0.8770694195943787

=> нет оснований отвергнуть гипотезу об адекватности модели

## j)

In [364]:
x_simpl = np.delete(x,3,0)

#b)
x_reg = x_simpl[:4,:].T
y_reg = x_simpl[4,:]
reg = LinearRegression().fit(x_reg, y_reg)
betas = reg.coef_

delta_boot = np.zeros((1000,4))
y = np.zeros((5,50))
for i in range(1000):
    y[0, :] = np.array(random.sample((x_simpl[0,:]).tolist(), N, counts = [100 for i in range(N)])) #хотим двумерную подвыборку или два цикла
    for m in range(50):
        for k in range(50):
            if x_simpl[0, k] == y[0, m]:
                y[:, m] = x_simpl[:, k]
                break
        
    x_boot = y[:4,:].T
    y_boot = y[4,:]
    reg_boot = LinearRegression().fit(x_boot, y_boot)
    betas_boot = reg_boot.coef_
    delta_boot[i, :] = np.abs(betas_boot - betas)

for i in range(4):
    m = sum(np.sort(delta_boot[:, i])>=abs(betas[i]))
    p_value = m/1000
    print(p_value)
    if p_value > 0.05:
        print('нет оснований отвергнуть гипотезу о незначимости коэффициента beta', i+1, sep='')
    else:
        print('отвергаем гипотезу о незначимости коэффициента beta', i+1, sep='')

0.0
отвергаем гипотезу о незначимости коэффициента beta1
0.0
отвергаем гипотезу о незначимости коэффициента beta2
0.003
отвергаем гипотезу о незначимости коэффициента beta3
0.002
отвергаем гипотезу о незначимости коэффициента beta4


In [365]:
#c)
TSS = np.sum(np.square(x_simpl[4] - np.mean(x_simpl[4])))
    
psi = np.ones((50,5))
psi[:,1:] = np.delete(x_simpl,4,0).T
F_inv = np.linalg.inv(psi.T@psi)
beta = F_inv@psi.T@x_simpl[4]
RSS1 = (x_simpl[4] - psi@beta).T @ (x_simpl[4] - psi@beta)
    
R_sqrt = 1 - RSS1/TSS

delta_est = R_sqrt**2/(1-R_sqrt**2) * (N - 5)/(5-1)
p_value = 1 - f.cdf(np.abs(delta_est), 5-1, N-1)
print('коэффициент детерминации: ', R_sqrt, sep='')
print('p_value: ', p_value, sep='')

коэффициент детерминации: 0.7236233239483509
p_value: 4.995221035697384e-07


=> отвергаем гипотезу о незначимости коэффициента детерминации

In [366]:
betas

array([ 2.9228148 , -1.85641035,  1.01967746, -1.22989382])

In [371]:
delta = (RSS1 - RSS)/RSS * (N-6)
p_value = 1 - f.cdf(delta,N-6,1)
p_value

0.19206696620648134

=> нет оснований отвергнуть гипотезу о том, что усложнение модели не является значимым

## k)

In [368]:
delta_boot = np.zeros(1000)
y = np.zeros((5,50))
for i in range(1000):
    y[0, :] = np.array(random.sample((x_simpl[0,:]).tolist(), N, counts = [100 for i in range(N)])) #хотим двумерную подвыборку или два цикла
    for m in range(50):
        for k in range(50):
            if x_simpl[0, k] == y[0, m]:
                y[:, m] = x_simpl[:, k]
                break
        
    TSS = np.sum(np.square(y[4] - np.mean(y[4])))
    
    psi = np.ones((50,5))
    psi[:,1:] = np.delete(y,4,0).T
    F_inv = np.linalg.inv(psi.T@psi)
    beta = F_inv@psi.T@y[4]
    RSS1 = (y[4] - psi@beta).T @ (y[4] - psi@beta)

    R_boot = 1 - RSS1/TSS
    delta_boot[i] = R_eta - R_boot

delta_est = R_eta - R_sqrt
m = sum(delta_boot >= delta_est)
p_value = m/1000
p_value

0.345

=> нет оснований отвергнуть гипотезу о том, что усложнение модели не является значимым